In [1]:
!pip install -q -U google-generativeai transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287

In [2]:
import os
from google.colab import userdata
import google.generativeai as genai
from transformers import pipeline

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [3]:
GOOGLE_API_KEY = userdata.get('gemini-api-key')  #API key for using Google AI studio
HF_TOKEN = userdata.get('emitrr-hf')    #API key for Huggingface transformers
genai.configure(api_key=GOOGLE_API_KEY)  #uses gemini-api-key for genai models (gemini-2.5-flash)

In [5]:
print("Initializing models... please wait.")

# Zero-shot classifier for Sentiment/Intent
classifier = pipeline("zero-shot-classification",model="facebook/bart-large-mnli",token=HF_TOKEN)

# Specialized Medical NER (Named Entity Recognition) model to identify symptoms, medications, and anatomy
ner_pipeline = pipeline("ner",model="d4data/biomedical-ner-all",aggregation_strategy="simple",token=HF_TOKEN)

Initializing models... please wait.


Device set to use cpu
Device set to use cpu


✅ Phase 1 Complete: Environment ready & models loaded.


In [6]:
#input data for the further NLP tasks
raw_transcript = """
Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.
Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
[Physical Examination Conducted]
Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
Patient: That’s a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
Patient: Thank you, doctor. I appreciate it.
Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
"""

In [8]:
import json

#
def get_medical_summary(transcript):
    model_id = 'gemini-2.5-flash'
    try:
        model = genai.GenerativeModel(model_id)

        #prompt to extract specific details from the transcript
        prompt = f"""
        You are a medical scribe. Extract the following details from the physician-patient transcript into a structured JSON format.

        Required Fields:
        - Patient_Name: Full name if available.
        - Symptoms: List of symptoms mentioned.
        - Diagnosis: The primary diagnosis mentioned by the doctor.
        - Treatment: List of treatments or medications mentioned.
        - Current_Status: The patient's current condition.
        - Prognosis: The expected outcome.

        Transcript:
        {transcript}

        Return ONLY valid JSON.
        """

        response = model.generate_content(prompt)
        return response.text

    except Exception as e:
        return f"Error: {e}. Try checking your API key permissions or model name."

# Ensure raw_transcript is defined (from previous step) and Execute
medical_json_output = get_medical_summary(raw_transcript)

# Print and Clean (Removing markdown backticks if Gemini adds them)
clean_json = medical_json_output.replace('```json', '').replace('```', '').strip()
print(clean_json)

# Convert to actual dictionary for future phases
medical_data = json.loads(clean_json)

{
  "Patient_Name": "Ms. Jones",
  "Symptoms": [
    "Discomfort (general)",
    "Head trauma (hit head on steering wheel)",
    "Neck pain",
    "Back pain",
    "Trouble sleeping (due to pain)",
    "Stiffness",
    "Occasional backaches"
  ],
  "Diagnosis": "Whiplash injury",
  "Treatment": [
    "Medical advice (from A&E)",
    "Painkillers",
    "Physiotherapy (10 sessions)"
  ],
  "Current_Status": "Doing better with occasional discomfort and backaches, not constant. No anxiety while driving or difficulty concentrating. Full range of movement in neck and back, no tenderness or signs of lasting damage. Muscles and spine in good condition. Back to usual routine after one week off work.",
  "Prognosis": "Expected to make a full recovery within six months of the accident. No signs of long-term damage or degeneration. No foreseen long-term impact on work or daily life."
}


In [9]:
# Code cell to process patient's statement to determine its sentiment and intent using zero-shot classification model

# The specific input from the assignment
patient_text = "I'm a bit worried about my back pain, but I hope it gets better soon."

# Sentiment Classification
sentiment_labels = ["Anxious", "Neutral", "Reassured"]
sentiment_result = classifier(patient_text, candidate_labels=sentiment_labels)

# Intent Detection
intent_labels = ["Seeking reassurance", "Reporting symptoms", "Expressing concern"]
intent_result = classifier(patient_text, candidate_labels=intent_labels)

# Structure the Output as JSON
sentiment_analysis = {
    "Sentiment": sentiment_result['labels'][0],
    "Intent": intent_result['labels'][0]
}

print(json.dumps(sentiment_analysis, indent=2))

{
  "Sentiment": "Anxious",
  "Intent": "Expressing concern"
}


In [17]:
# Code to convert medical transcipt into a SOAP note in JSON format

def get_final_soap_note(transcript):
    model = genai.GenerativeModel('gemini-2.5-flash')
    response = model.generate_content(f"Convert this to a SOAP JSON: {transcript}")
    return response.text

soap_json_raw = get_final_soap_note(raw_transcript)
print(soap_json_raw)

```json
{
  "SOAP": {
    "Subjective": {
      "ChiefComplaint": "Occasional backache and some discomfort following a car accident in September.",
      "HistoryOfPresentIllness": {
        "AccidentDetails": {
          "Date": "September 1st",
          "Time": "Around 12:30 PM",
          "Location": "Driving from Cheadle Hulme to Manchester",
          "Mechanism": "Stopped in traffic, hit from behind by another car, which pushed patient's car into the one in front.",
          "SeatbeltUse": "Yes, always worn."
        },
        "ImmediateSymptoms": [
          "Initial shock",
          "Hit head on steering wheel",
          "Pain in neck and back almost immediately"
        ],
        "InitialMedicalAttention": {
          "Facility": "Moss Bank Accident and Emergency",
          "DiagnosisAtAE": "Whiplash injury",
          "Imaging": "No X-rays done",
          "TreatmentAtAE": "Advice given, sent home"
        },
        "PostAccidentCourse": {
          "FirstFourWeeks": 